In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_community.chat_models import ChatOpenAI
import os
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chains import LLMChain
import prompts

In [2]:
# Generate theme 
from dataclasses import dataclass

@dataclass
class Song:    
    name:str
    moode:str 
    artist:str
    genre:str
    lyricks:str
    summary:str=None

    def info_for_summarization(self):
        return {
            "name":self.name,
            "moode":self.moode,
            "artist":self.artist,
            "genre":self.genre,
            "lyricks":self.lyricks
        }
    
    def to_dict(self):
        return {
            "summary":self.summary,
            **self.info_for_summarization()
        }


@dataclass
class VideoClip:
    idea:str=None
    visual_aesthetic:str=None
    concept:str=None
    scenario:str=None
    story_board:str=None    


@dataclass 
class ScenarioWriter:
    name: str
    memory: ConversationBufferMemory
    chat: ChatOpenAI
    video_clip:VideoClip
    ideas:list=None

    def summarize_song(self, song:Song):
        self.song=song
        res = self.chat.invoke([
            SystemMessage(
                content=prompts.SUMMARY_TEMPLATE
            ),
            HumanMessage(
                content=prompts.SONG_DATA_TEMPLATE.format(**song.info_for_summarization())
            ),
        ])
        song.summary = res.content
        return res.content

    def idea_generation(self, ideas_number:int):
        res = self.chat.invoke([
            SystemMessage(
                content=prompts.IDEAS_GENERATE_TEMPLATE
            ),
            HumanMessage(
                content=prompts.FULL_SONG_DATA_TEMPLATE.format(**self.song.to_dict())
            ),
        ])
        # print(res.content)
        self.ideas=res.content
        return res.content

    
    def choose_best_idea(self, ideas):
        res = self.chat.invoke([
            SystemMessage(
                content=prompts.CHOOSE_IDEA_TEMPLATE
            ),
            HumanMessage(
                content=prompts.IDEAS_TEMPLATE.format(**{"ideas":ideas})
            ),
            HumanMessage(
                content=prompts.SONG_DATA_TEMPLATE.format(**self.song.to_dict())
            ),
        ])
        self.video_clip.idea = res.content
        return res.content
    
    def concept_development(self):
        res = self.chat.invoke([
            SystemMessage(
                content=prompts.CONCEPT_GENERATE_TEMPLATE
            ),
            HumanMessage(
                content=prompts.SONG_DATA_TEMPLATE.format(**self.song.to_dict())
            ),
            HumanMessage(
                content=prompts.IDEA_TEMPLATE.format(**{"idea":self.video_clip.idea})
            ),
            HumanMessage(
                content=prompts.CONCEPT_TEMPLATE.format(**{"concept":self.video_clip.concept})
            ),
        ])
        self.video_clip.concept = res.content
        return res.content

    # def visual_aesthetic():
    #     pass

    def write_scenario(self):
        res = self.chat.invoke([
            SystemMessage(
                content=prompts.SCENARIO_GENERATE_TEMPLATE
            ),
            HumanMessage(
                content=prompts.SONG_DATA_TEMPLATE.format(**self.song.to_dict())
            ),
            HumanMessage(
                content=prompts.IDEA_TEMPLATE.format(**{"idea":self.video_clip.idea})
            ),
        ])
        self.video_clip.scenario = res.content
        return res.content

    def create_story_board():
        pass

### Song data

In [3]:
name = "Space in Between"
moode = "Sad"
artist = "Jan Blomquist"
genre = "Techno"
lyricks = """
[' (Verse 1)\n'
 'He walks through the world with a heavy heart,\n'
 "A burden that's hidden, deep within his pants,\n"
 'His soul cries out for more than just sex,\n'
 'But the weight of his package, never gives him a chance.\n'
 '\n'
 '(Chorus)\n'
 'Oh, the man with the big dick,\n'
 'Feels so small and so sad,\n'
 "He's just a man with a heavy heart,\n"
 'And a cross that he never had.\n'
 '\n'
 '(Verse 2)\n'
 "He's tired of the stares, the whispers, the jokes,\n"
 'The expectations, the pressure, the constant remarks,\n'
 'He wants to be loved for who he is inside,\n'
 'But his big dick, always steals the spotlight, leaving him in the dark.\n'
 '\n'
 '(Chorus)\n'
 'Oh, the man with the big dick,\n'
 'Feels so alone and misunderstood,\n'
 "He's just a man with a heavy heart,\n"
 "Living in a world that's so crude.\n"
 '\n'
 '(Bridge)\n'
 "He's more than just a body, more than just a tool,\n"
 "He's a person with feelings, with dreams and with rules,\n"
 'He wants to find love, he wants to find peace,\n'
 'But his big dick, only brings him release.\n'
 '\n'
 '(Chorus)\n'
 'Oh, the man with the big dick,\n'
 'Feels so unloved and unseen,\n'
 "He's just a man with a heavy heart,\n"
 "Drowning in a world that's so mean.\n"
 '\n'
 '(Outro)
 "So here's to the man with the big dick,"
 'May he find love, may he find his way,
 'May he be seen for who he truly is,
 'And not just the size of his package, at the end of the day.
"""
openapi_api_key=os.getenv('OPENAPI_API_KEY')

### Inits

In [4]:
song = Song(
    name=name,
    moode=moode,
    artist=artist,
    genre=genre,
    lyricks=lyricks,
)
video_clip=VideoClip()
scenario_writer = ScenarioWriter(
    name="Oleg", 
    memory = ConversationBufferMemory(), 
    chat = ChatOpenAI(
        model_name='gpt-4-0125-preview', 
        openai_api_key=openapi_api_key, 
        temperature=0.7
    ),
    video_clip=video_clip,
)

/home/alphapeas/.cache/pypoetry/virtualenvs/ai-models-ApbTBOR1-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
summary = scenario_writer.summarize_song(song)

summary

'"Space in Between" by Jan Blomquist is a melancholic track that delves into the emotional turmoil of a man burdened by societal perceptions related to his physical attributes. Set against a techno backdrop, the song explores themes of loneliness, misunderstanding, and the desire for genuine connection beyond physical appearances. Through a series of verses, choruses, and a bridge, the lyrics depict the protagonist\'s struggle with feeling unseen and unloved because of the focus on his physicality, highlighting his yearning to be appreciated for his inner self. The song aims to evoke empathy and understanding for individuals who are judged based on superficial qualities, urging a deeper recognition of their humanity.'

In [6]:
ideas = scenario_writer.idea_generation(5)

ideas

'["A grayscale world where everyone has visible, exaggerated physical traits symbolizing their personal burdens. The protagonist, marked by a glowing aura around his lower body, navigates this world, trying to connect beyond the superficial.",\n "A series of vignettes showing the protagonist engaging in everyday activities - shopping, walking in the park, sitting in a café - but always surrounded by a bubble that distorts and magnifies his midsection, illustrating his isolation and the barrier his physical trait creates.",\n "A dance performance in an empty, dimly lit warehouse, with the protagonist and others each dancing solo, expressing their longing and frustrations. As the song progresses, they come together, their movements synchronizing, symbolizing the search for understanding and connection.",\n "A narrative-driven concept following the protagonist through a day, from waking up alone in his apartment, going through his day facing whispers and stares, to finding solace in a sup

In [7]:
prompts.IDEAS_TEMPLATE.format(**{"ideas":ideas})

'Video Clip Ideas: ["A grayscale world where everyone has visible, exaggerated physical traits symbolizing their personal burdens. The protagonist, marked by a glowing aura around his lower body, navigates this world, trying to connect beyond the superficial.",\n "A series of vignettes showing the protagonist engaging in everyday activities - shopping, walking in the park, sitting in a café - but always surrounded by a bubble that distorts and magnifies his midsection, illustrating his isolation and the barrier his physical trait creates.",\n "A dance performance in an empty, dimly lit warehouse, with the protagonist and others each dancing solo, expressing their longing and frustrations. As the song progresses, they come together, their movements synchronizing, symbolizing the search for understanding and connection.",\n "A narrative-driven concept following the protagonist through a day, from waking up alone in his apartment, going through his day facing whispers and stares, to findi

In [8]:
best_idea = scenario_writer.choose_best_idea(ideas)
best_idea

'A narrative-driven concept following the protagonist through a day, from waking up alone in his apartment, going through his day facing whispers and stares, to finding solace in a support group where others share similar burdens, ending on a note of mutual understanding and acceptance.'

In [9]:
concept = scenario_writer.concept_development()
concept

'**30-Second Video Clip Concept for "Space in Between" by Jan Blomquist**\n\n**Opening Scene (0-5 seconds):**\nA close-up of an alarm clock ringing in a dimly lit room. The camera pans out to reveal our protagonist, a man in his late 20s, reluctantly waking up. His room is adorned with various personal items that hint at a deeper, unseen sadness.\n\n**Scene 2 (5-10 seconds):**\nA montage of clips shows the protagonist going through his day. Scenes include walking down the street while people around him whisper and give him odd looks, the camera focusing on their expressions and his discomfort. It’s clear the whispers aren’t positive, and they weigh heavily on him.\n\n**Scene 3 (10-15 seconds):**\nAt work, he’s the butt of a joke among colleagues; their laughter echoes as he walks away, visibly shrinking into himself. The focus is on his isolation in a crowded room.\n\n**Scene 4 (15-20 seconds):**\nA turning moment: he notices a flyer for a support group titled "Beyond Appearances". He 

In [10]:
# scenario = scenario_writer.write_scenario()
# scenario
res = scenario_writer.chat.invoke([
            SystemMessage(
                content=prompts.SCENARIO_GENERATE_TEMPLATE
            ),
            HumanMessage(
                content=prompts.SONG_DATA_TEMPLATE.format(**scenario_writer.song.to_dict())
            ),
            HumanMessage(
                content=prompts.IDEA_TEMPLATE.format(**{"idea":scenario_writer.video_clip.idea})
            ),
        ])

'**Space in Between - 30 Second Video Clip Scenario**\n\n**0-3 seconds:** A close-up of an alarm clock in a dimly lit room, the time switching to 7:00 AM, followed by a hand reaching out to turn it off. The camera pans to show the protagonist lying in bed, staring at the ceiling with a sense of dread.\n\n**4-6 seconds:** Cut to the protagonist standing in front of a mirror, getting dressed. The camera focuses on his troubled expression, avoiding explicit visuals but implying the source of his distress.\n\n**7-9 seconds:** The protagonist exits his apartment, head down. People on the street glance his way, whispering and nudging each other, their expressions a mix of curiosity and mockery.\n\n**10-12 seconds:** A quick montage of the day\'s encounters: a coworker making a crude joke, a stranger on the bus staring, and a disappointing date that ends awkwardly, all highlighting his isolation and discomfort.\n\n**13-15 seconds:** The protagonist, looking deflated, notices a flyer on a comm